In [ ]:
!pip install sovai[full]

## Clinical Trials

In [4]:
import sovai as sov

sov.token_auth(token="visit https://sov.ai/profile for your token")

Prediction Data

In [5]:
df_clinical = sov.data("trials/predict", full_history=True) 

In [6]:
df_clinical["links"] = "https://clinicaltrials.gov/study/" + df_clinical["trial_id"]

In [9]:
df_clinical.head()

,date,ticker,trial_id,lead_sponsor_name,official_title,phase_category,success_prediction,economic_effect,duration_prediction,success_composite,links
0,2010-08-12,A,NCT01181479,Agile Therapeutics,"Open-Label, Randomized, Parallel Group, Study ...",phase_3,0.970,0.630,2882.000,0.841,https://clinicaltrials.gov/study/NCT01181479
1,2010-11-05,A,NCT01236768,Agile Therapeutics,"An Open-Label, Randomized, Phase 3 Study Of Th...",phase_3,0.950,0.670,2506.000,0.846,https://clinicaltrials.gov/study/NCT01236768
2,2010-11-16,A,NCT01243580,Agile Therapeutics,An Open-Label Study To Evaluate Pharmacokineti...,phase_1,0.960,0.630,2590.000,0.793,https://clinicaltrials.gov/study/NCT01243580
3,2010-11-29,A,NCT01250210,Agile Therapeutics,Evaluation Of Pharmacodynamic Effects On Ovula...,phase_2,0.950,0.700,1084.000,0.850,https://clinicaltrials.gov/study/NCT01250210
4,2011-06-13,A,NCT01375946,Agile Therapeutics,A Pharmacokinetic And Wearability Study Of The...,phase_1,0.940,0.630,2383.000,0.783,https://clinicaltrials.gov/study/NCT01375946


Trial Information

In [10]:
df_describe = sov.data("trials/describe", full_history=True)

In [11]:
df_describe.tail()

,ticker,date,trial_id,official_title,sponsor_study_id,brief_title,lead_sponsor_name,sponsor_type,phase_category,study_type,enrollment_count,number_of_locations,healthy_volunteers,enrollment_type,study_size_category,condition_keywords,primary_condition,intervention_type,primary_intervention,intervention_name,intervention_arm_group_labels,intervention_description,has_data_monitoring_committee,responsible_party_investigator_affiliation,responsible_party_investigator_title,responsible_party_investigator_name,first_posted_date,last_update_posted_date,start_date,study_completion_date,primary_completion_date,study_locations_city,study_locations_country,study_locations_geopoint,study_locations_facility,study_locations_zip,study_locations_state,standard_age_groups,sex,minimum_age,maximum_age,overall_status,status_category,status_verified_date,primary_outcomes_measures,primary_outcomes_timeframes,primary_outcomes_descriptions,secondary_outcomes_measures,secondary_outcomes_timeframes,secondary_outcomes_descriptions,has_results,conditions,brief_summary,detailed_description,masking,allocation,intervention_model,primary_purpose,has_expanded_access,study_duration_days,trial_duration,references_type,references_citation,references_pmid,collaborators_name,collaborators_class,ipd_sharing,success_prediction,economic_effect,duration_prediction,success_composite
513998,ZYXI,2021-08-06,NCT05012462,Clinical Evaluation Of The Cm-1500 During Aphe...,ZMS-1500-2021-Apheresis,Clinical Evaluation Of The Cm-1500 During Aphe...,Zynex Monitoring Solutions,INDUSTRY,other,INTERVENTIONAL,75,1,True,ACTUAL,Small,nan,Fluid Loss,DEVICE,CM-1500,CM-1500,None,The Cm-1500 Is A U.S. Fda Cleared Non-Invasive...,False,None,None,None,2021-08-19,2022-12-21,2021-09-20,2022-12-20,2022-12-20,Denver,United States,"lat: 39.73915, lon: -104.9847",Vitalant Blood Donation,80230,Colorado,ADULT; OLDER_ADULT,ALL,18,<NA>,COMPLETED,completed,2022-12-01,Characterize changes in the Relative Index dur...,1-6 hours,The primary objective of the study is to chara...,Characterize changes in blood pressure relativ...,1-6 hours; 1-6 hours; 1-6 hours; 1-6 hours,Characterize changes in blood pressure that oc...,False,Fluid Loss,"The Study Is A Prospective, Single-Arm, Non-Ra...",None,NONE,NA,SINGLE_GROUP,DIAGNOSTIC,False,456,456.000,None,None,None,None,None,NO,0.270,0.080,456.000,0.211
513999,ZYXI,2021-11-08,NCT05125848,Clinical Evaluation Of The Cm-1500 During Hemo...,ZMS-1500-2021-Dialysis,Clinical Evaluation Of The Cm-1500 During Hemo...,Zynex Monitoring Solutions,INDUSTRY,other,INTERVENTIONAL,15,1,False,ACTUAL,Small,nan,Fluid Loss,DEVICE,CM-1500,CM-1500,Hemodialysis Single Group Assignment,The Cm-1500 Is A U.S. Fda Cleared Non-Invasive...,False,None,None,None,2021-11-18,2024-05-01,NaT,NaT,NaT,Minneapolis,United States,"lat: 44.97997, lon: -93.26384",DaVita Clinical Research,55404,Minnesota,ADULT; OLDER_ADULT,ALL,18,<NA>,COMPLETED,completed,2024-04-01,Average Relative Index Value Following a Hemod...,Duration of post-hemodialysis recovery period ...,The primary objective is to characterize the a...,Mean Intrasubject Variability (Standard Deviat...,Duration of standard dialysis procedure (Range...,The secondary objective is to characterize int...,True,Fluid Loss; Dialysis,"The Study Is A Prospective, Single-Arm, Non-Ra...",None,NONE,NA,SINGLE_GROUP,DIAGNOSTIC,False,<NA>,NaN,None,None,None,None,None,None,0.560,0.290,905.000,0.443
514000,ZYXI,2022-07-14,NCT05463198,Evaluation Of Physiological Variables And Dete...,ZMS-1600-Blood Loss Detection,Evaluation Of Physiological Variables And Dete...,Zynex Monitoring Solutions,INDUSTRY,other,INTERVENTIONAL,351,4,True,ACTUAL,Medium,nan,Fluid Loss,DEVICE,CM-1600,CM-1600,Blood Loss,Subjects Undergoing Blood Donation Will Be Con...,False,None,None,None,2022-07-18,2024-08-13,2022-09-14,2023-05-26,2023-04-27,Denver; Bradenton; Columbus; Chattanooga,United States; United States; United States; U...,"lat: 39.73915, lon: -104.9847; lat: 27.49893, ...",Vitalant Rese

Specific Tickers Select

In [ ]:
df_pfizer = sov.data("trials/predict", tickers=["PFE"]) 

In [4]:
df_clinical.sort_values("date")

,date,ticker,trial_id,lead_sponsor_name,official_title,phase_category,success_prediction,economic_effect,duration_prediction,success_composite
513354,1999-10-18,WINT,NCT00004500,Windtree Therapeutics,"A Multicenter, Randomized, Controlled Trial Co...",phase_3,0.060,0.490,1841.000,0.279
512527,1999-10-18,UTHR,NCT00004497,United Therapeutics,None,phase_3,0.950,0.600,5637.000,0.805
32118,1999-10-29,NVS,NCT00000174,Novartis,None,phase_3,0.920,0.490,2065.000,0.770
36726,1999-11-01,PFE,NCT00003343,Pfizer,"A Randomized, Double-Blind, Placebo-Controlled...",phase_3,0.300,0.170,4664.000,0.290
36723,1999-11-01,PFE,NCT00003720,Pfizer,"A Multicenter, Dose Escalating Study In Patien...",phase_1,0.910,0.600,4679.000,0.742
...,...,...,...,...,...,...,...,...,...,...
20808,2024-10-25,GSK,NCT06663436,Glaxosmithkline,"A Phase 2 Single-Blind, Randomized, Controlled...",phase_2,0.840,0.430,327.000,0.704
32829,2024-10-27,NVS,NCT06662812,Novartis,Disease Characteristics And Treatment Patterns...,not_specified,0.270,0.070,730.000,0.204
32830,2024-10-27,NVS,NCT06662825,Novartis,Seg-2022-01 Patient Characteristics And Treatm...,not_specified,0.310,0.070,323.000,0.233
511796,2024-10-28,TEVA,NCT06664619,"Teva Branded Pharmaceutical Products R&D, Inc.","A 4-Week, Double-Blind, Placebo-Controlled, Pa...",phase_3,0.850,0.440,560.000,0.727


In [ ]:
#https://clinicaltrials.gov/study/NCT06628310